In [1]:
# Cell 0 — imports and paths
import os, math, time, pickle
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

PROJECT_ROOT = Path(r"C:\Users\aibel\Desktop\Heizel Ann Joseph\Parkinsons Disease")
DATA_ROOT = PROJECT_ROOT / "data" / "PaHaW_dataset"
PAHAW_PUBLIC = DATA_ROOT / "PaHaW_public"
META_FILE = DATA_ROOT / "PaHaW_files" / "corpus_PaHaW.xlsx"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("PAHAW_PUBLIC exists:", PAHAW_PUBLIC.exists())
print("Processed dir:", PROCESSED_DIR)


PROJECT_ROOT: C:\Users\aibel\Desktop\Heizel Ann Joseph\Parkinsons Disease
PAHAW_PUBLIC exists: True
Processed dir: C:\Users\aibel\Desktop\Heizel Ann Joseph\Parkinsons Disease\data\processed


In [2]:
# Cell 1 — helper functions (run this BEFORE the big builder)
import numpy as np
import pandas as pd
import math
from scipy.signal import savgol_filter, find_peaks
from scipy.optimize import curve_fit

def load_svc(path):
    rows = []
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        lines = [ln.rstrip('\n') for ln in f if ln.strip()!='']
    if len(lines) < 2:
        return pd.DataFrame(columns=["x","y","time","pen","azim","alt","press"])
    # skip header if first line looks like a small count
    start_idx = 1 if len(lines[0].split()) <= 2 else 0
    for ln in lines[start_idx:]:
        parts = ln.split()
        if len(parts) < 7:
            # drop leading index tokens like "00:" if present
            if parts and parts[0].endswith(':'):
                parts = parts[1:]
        if len(parts) >= 7:
            parts = parts[-7:]
        if len(parts) != 7:
            continue
        try:
            x, y, ts, pen, az, alt, pr = parts
            rows.append([float(x), float(y), float(ts), float(pen), float(az), float(alt), float(pr)])
        except Exception:
            continue
    df = pd.DataFrame(rows, columns=["x","y","time","pen","azim","alt","press"])
    return df

def preprocess_df(df, smoothing_window=7, smoothing_poly=2):
    df = df.copy().reset_index(drop=True)
    if len(df) == 0:
        return df
    df['y'] = df['y'] - df['y'].mean()
    df['time_s'] = df['time'] - df['time'].iloc[0]
    df['dt'] = df['time_s'].diff().fillna(1/1000).replace(0, 1/1000)
    if len(df) >= smoothing_window:
        df['x_s'] = savgol_filter(df['x'], smoothing_window, smoothing_poly)
        df['y_s'] = savgol_filter(df['y'], smoothing_window, smoothing_poly)
    else:
        df['x_s'] = df['x']
        df['y_s'] = df['y']
    df['vx'] = df['x_s'].diff().fillna(0) / df['dt']
    df['vy'] = df['y_s'].diff().fillna(0) / df['dt']
    df['speed'] = np.sqrt(df['vx']**2 + df['vy']**2)
    df['ax'] = df['vx'].diff().fillna(0) / df['dt']
    df['ay'] = df['vy'].diff().fillna(0) / df['dt']
    df['accel'] = np.sqrt(df['ax']**2 + df['ay']**2)
    with np.errstate(divide='ignore', invalid='ignore'):
        num = df['vx']*df['ay'] - df['vy']*df['ax']
        den = (df['vx']**2 + df['vy']**2)**1.5
        df['curvature'] = np.abs(num) / (den + 1e-12)
    df['curvature'] = df['curvature'].fillna(0)
    return df

def segment_by_pen(df):
    pen = df['pen'].values
    strokes = []
    start = None
    for i, p in enumerate(pen):
        if p == 1 and start is None:
            start = i
        if p == 0 and start is not None:
            strokes.append((start, i-1))
            start = None
    if start is not None:
        strokes.append((start, len(df)-1))
    return strokes

def split_stroke_by_speed_minima(df, s, e, prom=0.05, dist=8, min_points=6):
    seg = df.iloc[s:e+1].reset_index(drop=True)
    speed = seg['speed'].to_numpy()
    inv = -speed
    peaks, _ = find_peaks(inv, prominence=prom, distance=dist)
    if len(peaks)==0:
        intervals = [(s,e)]
    else:
        cuts = [s + int(p) for p in peaks]
        intervals = []
        prev = s
        for c in cuts:
            intervals.append((prev, c))
            prev = c+1
        if prev <= e:
            intervals.append((prev, e))
    intervals = [iv for iv in intervals if (iv[1]-iv[0]+1) >= min_points]
    if len(intervals)==0:
        intervals = [(s,e)]
    return intervals

def split_all_strokes(df, pen_strokes, prom=0.05, dist=8, min_points=6):
    subs = []
    for (s,e) in pen_strokes:
        parts = split_stroke_by_speed_minima(df, s, e, prom=prom, dist=dist, min_points=min_points)
        subs.extend(parts)
    return subs

# Beta & ellipse helpers
def beta_velocity(t, A, t0, t1, a, b):
    t = np.array(t)
    v = np.zeros_like(t, dtype=float)
    denom = (t1 - t0) if (t1 - t0) != 0 else 1e-8
    u = (t - t0) / denom
    mask = (u > 0) & (u < 1)
    uu = u[mask]
    with np.errstate(all='ignore'):
        v_mask = A * (uu**a) * ((1 - uu)**b)
    v[mask] = v_mask
    return v

def fit_beta_to_substroke(df, s, e, min_points=8):
    seg = df.iloc[s:e+1].reset_index(drop=True)
    if len(seg) < min_points:
        return None
    t = seg['time_s'].to_numpy()
    speed = seg['speed'].to_numpy()
    A0 = max(np.max(speed), 1e-3)
    t0_0 = t[0]
    t1_0 = t[-1]
    p0 = [A0, t0_0, t1_0, 2.0, 2.0]
    bounds = ([0.0, t[0]-0.5, t[-1]-1.0, 0.05, 0.05],
              [A0*20+1.0, t[0]+0.5, t[-1]+0.5, 12.0, 12.0])
    try:
        popt, _ = curve_fit(beta_velocity, t, speed, p0=p0, bounds=bounds, maxfev=20000)
        return {"A":float(popt[0]), "t0":float(popt[1]), "t1":float(popt[2]), "a":float(popt[3]), "b":float(popt[4])}
    except Exception:
        return None

def fit_ellipse_least_squares(xs, ys):
    x = xs[:,None]; y = ys[:,None]
    D = np.hstack([x*x, x*y, y*y, x, y, np.ones_like(x)])
    try:
        U, S, Vt = np.linalg.svd(D, full_matrices=False)
        a = Vt.T[:,-1]
        A, B, Cc, Dd, E, Ff = a.flatten()
    except Exception:
        return None
    denom = B*B - 4*A*Cc
    if np.isclose(denom, 0):
        return None
    x0 = (2*Cc*Dd - B*E) / denom
    y0 = (2*A*E - B*Dd) / denom
    up = 2*(A*E*E + Cc*Dd*Dd + Ff*B*B - 2*B*Dd*E - A*Cc*Ff)
    tmp = (A - Cc)**2 + B*B
    sqrt_tmp = np.sqrt(np.abs(tmp))
    down1 = (denom) * ((Cc - A) + sqrt_tmp)
    down2 = (denom) * ((Cc - A) - sqrt_tmp)
    if down1 == 0 or down2 == 0:
        return None
    try:
        a_len = np.sqrt(np.abs(up / down1))
        b_len = np.sqrt(np.abs(up / down2))
    except Exception:
        return None
    if b_len > a_len:
        a_len, b_len = b_len, a_len
    theta = 0.5 * np.arctan2(B, (A - Cc))
    ecc = np.sqrt(max(0, 1 - (b_len**2)/(a_len**2))) if a_len!=0 else 0.0
    return {"xc":float(x0), "yc":float(y0), "a":float(a_len), "b":float(b_len), "angle_rad":float(theta), "ecc":float(ecc)}

print("Helper functions loaded — ready to run the sequence builder (Cell 2).")

Helper functions loaded — ready to run the sequence builder (Cell 2).


In [3]:
# Cell 2 — generate sequences for ALL .svc files (BLSTM-ready)
import json, gc, pickle
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm   # plain tqdm (no notebook mode)

# --- Load metadata ---
meta = pd.read_excel(META_FILE)
meta['ID'] = meta['ID'].astype(str).str.zfill(5)
meta_map = dict(zip(meta['ID'], meta['Disease']))  # "PD" or "H"

# --- List files ---
svc_paths = sorted(PAHAW_PUBLIC.rglob("*.svc"))
print("Total svc files:", len(svc_paths))

# --- Storage ---
all_seqs = []     # list of (n_substrokes, feature_dim) arrays
all_labels = []   # PD=1, H=0 per file
all_meta = []     # store {"path","subject"}
failed = []

# --- Resume feature ---
partial_file = PROCESSED_DIR / "all_sequences_partial.pkl"

if partial_file.exists():
    print("Resuming from partial file...")
    with open(partial_file, "rb") as f:
        data = pickle.load(f)
    all_seqs = data.get("seqs", [])
    all_labels = data.get("labels", [])
    all_meta = data.get("meta", [])
    processed_paths = set(data.get("paths", []))
else:
    processed_paths = set()

print("Already processed:", len(processed_paths))

# --- Feature vector for a single substroke ---
def compute_substroke_feature_vector(seg, df_local, s_abs, e_abs):
    duration = seg['time_s'].iloc[-1] - seg['time_s'].iloc[0]
    amp_x = seg['x_s'].max() - seg['x_s'].min()
    amp_y = seg['y_s'].max() - seg['y_s'].min()
    amplitude = math.sqrt(amp_x**2 + amp_y**2)
    mean_speed = seg['speed'].mean()
    mean_press = seg['press'].mean()

    # beta & ellipse
    beta = fit_beta_to_substroke(df_local, s_abs, e_abs)
    ell  = fit_ellipse_least_squares(seg['x_s'].to_numpy(),
                                     seg['y_s'].to_numpy())

    beta_A = beta['A'] if isinstance(beta, dict) else np.nan
    beta_a = beta['a'] if isinstance(beta, dict) else np.nan
    beta_b = beta['b'] if isinstance(beta, dict) else np.nan

    ell_a  = ell['a'] if isinstance(ell, dict) else np.nan
    ell_b  = ell['b'] if isinstance(ell, dict) else np.nan
    ell_ecc = ell['ecc'] if isinstance(ell, dict) else np.nan

    # simple fuzzy: compare against global medians
    f_speed_high = 1.0 if mean_speed > np.nanmedian(df_local['speed']) else 0.0
    f_press_high = 1.0 if mean_press > np.nanmedian(df_local['press']) else 0.0
    f_curv_high = 1.0 if seg['curvature'].mean() > np.nanmedian(df_local['curvature']) else 0.0

    return np.array([
        duration, amplitude, mean_speed, mean_press,
        beta_A, beta_a, beta_b,
        ell_a, ell_b, ell_ecc,
        f_speed_high, f_press_high, f_curv_high
    ], dtype=float)


# --- MAIN LOOP ---
new_files = 0

for p in tqdm(svc_paths, desc="Processing SVC files"):
    pstr = str(p)

    # skip already processed
    if pstr in processed_paths:
        continue

    try:
        df_local = load_svc(p)
        if len(df_local) < 5:
            failed.append({"path": pstr, "reason": "too short"})
            processed_paths.add(pstr)
            continue

        df_local = preprocess_df(df_local)
        pen_strokes = segment_by_pen(df_local)
        subs = split_all_strokes(df_local, pen_strokes, prom=0.05, dist=8, min_points=6)

        feats = []
        for (s, e) in subs:
            seg = df_local.iloc[s:e+1].reset_index(drop=True)
            if len(seg) < 6:
                continue
            vec = compute_substroke_feature_vector(seg, df_local, s, e)
            feats.append(vec)

        if len(feats) == 0:
            failed.append({"path": pstr, "reason": "no substrokes"})
            processed_paths.add(pstr)
            continue

        seq_arr = np.vstack(feats)
        all_seqs.append(seq_arr)

        # label based on subject ID
        subject = p.parent.name
        label = 1 if meta_map.get(subject, "H") == "PD" else 0
        all_labels.append(label)

        all_meta.append({"path": pstr, "subject": subject})

        processed_paths.add(pstr)
        new_files += 1

        # periodic saving every 25 files
        if new_files % 25 == 0:
            with open(partial_file, "wb") as f:
                pickle.dump({
                    "seqs": all_seqs,
                    "labels": all_labels,
                    "meta": all_meta,
                    "paths": list(processed_paths)
                }, f)
            print(f"\nSaved partial progress after {new_files} new files.")

    except Exception as e:
        failed.append({"path": pstr, "reason": str(e)})
        processed_paths.add(pstr)
        continue


# --- Final save ---
np.savez(PROCESSED_DIR / "all_sequences.npz",
         sequences=all_seqs,
         labels=all_labels,
         meta=all_meta)

with open(PROCESSED_DIR / "all_sequences_failed.pkl", "wb") as f:
    pickle.dump({"failed": failed}, f)

print("\nFinished!")
print("Total files attempted:", len(svc_paths))
print("Total sequences:", len(all_seqs))
print("Failed:", len(failed))
print("Saved to:", PROCESSED_DIR / "all_sequences.npz")

Total svc files: 597
Resuming from partial file...
Already processed: 25


Processing SVC files:   8%|▊         | 50/597 [01:18<50:10,  5.50s/it]  


Saved partial progress after 25 new files.


Processing SVC files:  13%|█▎        | 75/597 [02:05<15:21,  1.77s/it]


Saved partial progress after 50 new files.


...

Processing SVC files:  96%|█████████▋| 575/597 [15:07<00:52,  2.39s/it]


Saved partial progress after 550 new files.


Processing SVC files: 100%|██████████| 597/597 [15:50<00:00,  1.59s/it]


Finished!
Total files attempted: 597
Total sequences: 597
Failed: 0
Saved to: C:\Users\aibel\Desktop\Heizel Ann Joseph\Parkinsons Disease\data\processed\all_sequences.npz



c:\Users\aibel\Desktop\Heizel Ann Joseph\Parkinsons Disease\.venv\lib\site-packages\numpy\lib\npyio.py:716: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)
